In [ ]:
import pandas as pd
import numpy as np
#pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.max_columns', None)  


In [ ]:
def makedata(FILENAME):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    print(data)

    #reshape the data so it will fit together. 
    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    
    #this puts each full game box score on the same line. Originally it had
    #every 2 rows represent the home and road team, gotta put it on one. 
    #Unclear if this gives any information to the neural net, maybe it does if its just one 1 or 0 not 2.
    
    #the way to make it just one is to "forget" 2 columns, will name those 2 in a sec. 
    dat = []
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    dat = np.array(dat)    
    
   # if show:
      #  print(data)
    #separate X and y, while hot encoding the winner and loser too. 
    y = []
    for j in range(len(dat)):
        home = dat[j,8]
        road = dat[j,20]
       # print(home,road)
    #distinguish home win or road win. 
        if home > road:
            y.append(np.array([0,1]))  # 0 1 mean home won
        else:
            y.append(np.array([1,0]))  #  1 0 means away won. 
    y = np.array(y)  #must be an array!
    #now split up that big guy to get X
    X1 = np.concatenate((dat[:,0:10],dat[:,11:21]),axis=1)
    X = np.concatenate((X1,dat[:,22:28]),axis=1)   
    return X,y

In [ ]:
X , y = makedata('1617_boxscores.csv')

In [ ]:
X[0]

In [ ]:
0     0.367816  0.333333  13  29   42  17  22   6  18   6   88  0.750000


In [ ]:
data

Data:
1. Road Orbs
2. Road Drbs
3. Road Trbs
4. Road Asts
5. Road Pfs
6. Road Stls
7. Road Tos. 
8. Road Blks 
9. Road 3p%
10. Road Fg%
11. Road Ft% 
12. Road venue Home (0)  REMOVED
13. Road venue Away (1) REMOVED
14. Home Orbs
15. Home Drbs
16. Home Trbs
17. Home Asts
18. Home Pfs
19. Home Stls
20. Home Tos. 
21. Home Blks 
22. Home 3p%
23. Home Fg%
24. Home Ft% 
25. Home venue Home (0) REMOVED
26. Home venue Away (1) REMOVED

With Outputs 

27. Home team win (1)
28. Road team win (0)


So you enter the statistics of both teams, and it gives a classification of whether or not the home or away team wins.
Unclear how good this actually is at characterizing teams. Should use team stats from as recent a metric as possible to reflect current gameplay


In [ ]:
from sklearn import preprocessing, cross_validation, neighbors

X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.4)

In [ ]:
np.shape(X_train)

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.verbose =1
model.batch_size = 20
model.n_layers_ = 50
model.n_outputs_= 50

In [ ]:
model.fit(X_train,y_train)

In [ ]:
X_train[0]

In [ ]:
np.set_printoptions(threshold=np.nan)

model.score(X_test,y_test)  

Let's do a raw version where I pull the last 5 stats of wiz vs pels 

In [ ]:
model.predict(X_test)

In [ ]:
data = pd.read_csv('predictionstuff.csv')
data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
del data['3P'],data['3PA']
data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
del data['FG'],data['FGA']
data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
del data['FT'],data['FTA'], data['2P'],data['2PA']
data['ORB']= np.divide(data['ORB'].values,data['G'].values)
data['DRB']= np.divide(data['DRB'].values,data['G'].values)
data['TRB']= np.divide(data['TRB'].values,data['G'].values)
data['TOV']= np.divide(data['TOV'].values,data['G'].values)
data['BLK']= np.divide(data['BLK'].values,data['G'].values)
data['AST']= np.divide(data['AST'].values,data['G'].values)
data['STL']= np.divide(data['STL'].values,data['G'].values)
data['PF']= np.divide(data['PF'].values,data['G'].values)
del data['G']
teams = data['Tm']
del data['Tm']

In [ ]:
#data = data[['ORB','DRB','TRB','AST','STL','BLK','TOV','PF','3P%','FG%','FT%']]
data = data[[  'FG%'  ,'3P%' , 'ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK'  ,'FT%']]
#team data is now lined up to be easily concatenated. 

In [ ]:
lakers = data.values[21,:]
nugs = data.values[18,:]
kings = data.values[27,:]
magic = data.values[25,:]
cavs = data.values[17,:]
clips = data.values[6,:]
chi = data.values[16,:]
det = data.values[4,:]
bucks = data.values[8,:]
por = data.values[12,:]
wash = data.values[29,:]
knicks = data.values[24,:]
utah = data.values[14,:]
grizz = data.values[7,:]
atl = data.values[0,:]
pacers = data.values[20,:]
houston = data.values[5,:]
raptors = data.values[28,:]
heat = data.values[22,:]
sixers = data.values[11,:]
bos = data.values[15,:]
wolves = data.values[23,:]
nets = data.values[1,:]
hornets = data.values[2,:]
thunder = data.values[10,:]
suns = data.values[26,:]
spurs =data.values[13,:]
dubs = data.values[19,:]
pels = data.values[9,:]

In [ ]:
#omg1 = np.concatenate(ROADTEAM,HOMETEAM)

ok = np.concatenate((suns,dubs))
model.predict(np.array([ok]))

In [ ]:
nets

In [ ]:
games = np.array(np.concatenate((ROADTEAM,HOMETEAM)),)

In [ ]:
games = np.array([np.concatenate((chi,det)),np.concatenate((dubs,por)),np.concatenate((suns,nets)),np.concatenate((houston,raptors)),np.concatenate((utah,grizz)),np.concatenate((knicks,bucks)),np.concatenate((lakers,nugs)),np.concatenate((magic,kings)),np.concatenate((wash,pels)),np.concatenate((cavs,clips))])

In [ ]:
games

In [ ]:
model.predict(games)

3/8 Games and if it was correct:

Suns @ OKC YES

Nets @ HOR NO

Bos @ Min NO

Phi @ Mia YES

Sas @ GSW Says spurs, 

This is a good start. What's left?

1. What about assists, rebounds, steals, etc.? Can a table do that too? 

2. Hot encode wins and losses. Need to move that result column and delete everyhting except W and L.

3. Normalize data?

4. Create a net

5. Figure out prediction data, go by season averages? L5 averages? 

In [ ]:
data = pd.read_csv('expensive.csv',nrows=52) #nrows 52 because reader going to far otherwise. 

In [ ]:
data = pd.get_dummies(data)

In [ ]:
model.predict

Remove unneccesary stats. Flatten 2 adjascent rows because they're the same game!

In [ ]:
Xfirst = np.concatenate((dat[:,0:8],dat[:,9:19]),axis=1) # + dat[:,20:22]

In [ ]:
X = np.concatenate((Xfirst,dat[:,20:22]),axis=1)

In [ ]:
np.shape(dat[:,0:8])

In [ ]:
np.shape(X)

In [ ]:
y =  

In [ ]:
type(X)

In [ ]:
type(y)

In [ ]:
np.shape(y)

In [ ]:
 np.shape(X)

In [ ]:

import numpy as np
from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.4)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.verbose =1
model.batch_size = 2
model.n_layers_ = 100

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

In [ ]:
pred = X_test[0]

In [ ]:
np.shape(pred)

In [ ]:
model.predict(np.array([pred]))

In [ ]:
y_test[0]